In [1]:
import numpy as np

environment_rows = 11
environment_columns = 11
actions = ['up', 'right', 'down', 'left']

In [2]:
# Create 2D array to hold rewards for each state
rewards = np.full((environment_rows, environment_columns), -100.)
rewards[0, 5] = 100.

aisles = {}
aisles[1] = [i for i in range(1, 10)]
aisles[2] = [1, 7, 9]
aisles[3] = [i for i in range(1, 8)]
aisles[3].append(9)
aisles[4] = [3, 7]
aisles[5] = [i for i in range(11)]
aisles[6] = [5]
aisles[7] = [i for i in range(1, 10)]
aisles[8] = [3, 7]
aisles[9] = [i for i in range(11)]

for row_index in range(1, 10):
    for column_index in aisles[row_index]:
        rewards[row_index, column_index] = -1

print('Rewards for each state:')
for row in rewards:
    print(row)

Rewards for each state:
[-100. -100. -100. -100. -100.  100. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100.   -1. -100. -100. -100. -100. -100.   -1. -100.   -1. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100.   -1. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.]


In [3]:
# initialize V value at each state
V = np.zeros((environment_rows, environment_columns, 1))
V[:, :, 0] = rewards

In [4]:
# Print initialized values V
print(V[:,:,0])

[[-100. -100. -100. -100. -100.  100. -100. -100. -100. -100. -100.]
 [-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
 [-100.   -1. -100. -100. -100. -100. -100.   -1. -100.   -1. -100.]
 [-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.   -1. -100.]
 [-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
 [  -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
 [-100. -100. -100. -100. -100.   -1. -100. -100. -100. -100. -100.]
 [-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
 [-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
 [  -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
 [-100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.]]


In [5]:
# initialize policy at each state
optimal_policy = np.zeros((environment_rows, environment_columns, 1))

In [6]:
# define a function that determines if the specified location is a terminal state
def is_terminal_state(current_row_index, current_column_index):
    if rewards[current_row_index, current_column_index] == -1.:
        return False
    else:
        return True

In [7]:
# define a function that will get the next location based on the chosen action
def get_next_location(current_row_index, current_column_index, action_index):
    new_row_index = current_row_index
    new_column_index = current_column_index
    if actions[action_index] == 'up' and current_row_index > 0:
        new_row_index -= 1
    elif actions[action_index] == 'right' and current_column_index < environment_columns - 1:
        new_column_index += 1
    elif actions[action_index] == 'down' and current_row_index < environment_rows - 1:
        new_row_index += 1
    elif actions[action_index] == 'left' and current_column_index > 0:
        new_column_index -= 1
    return new_row_index, new_column_index

In [8]:
epsilon = 0.9 # percentage of time to take the best action (instead of a random action)
pr_best = epsilon + (1 - epsilon) / 4 # probability of moving to the best adjacent location
pr_others = (1 - epsilon) / 4 # probability of moving to the adjacent locations other than the best
gamma = 0.9 # discount factor

In [9]:
def value_iterations(V):
    number_iterations = 0
    while True:
        # Store current values of states
        oldV = V.copy()
        # Iterate and update value and optimal policy of each state
        for row_index in range(0,11):
            for column_index in range(0,11):
                if not is_terminal_state(row_index, column_index):
                    Q = {}
                    # Find value of taking each action a in current state s = (row_index, column_index) and store it in Q
                    for a in range(0,4):
                        new_row_index, new_column_index = get_next_location(row_index, column_index, a)

                        #find out the other 3 non-favorable directions
                        direction_index = np.array([0,1,2,3])
                        other_direction_index = np.delete(direction_index, np.where(direction_index == a))
                        other_row_index_1, other_column_index_1 = get_next_location(row_index, column_index, other_direction_index[0])
                        other_row_index_2, other_column_index_2 = get_next_location(row_index, column_index, other_direction_index[1])
                        other_row_index_3, other_column_index_3 = get_next_location(row_index, column_index, other_direction_index[2])

                        # Compute Q[a] for the current state s = (row_index, column_index) and action a.
                        Q[a] = rewards[new_row_index, new_column_index] + gamma * (pr_best * oldV[new_row_index, new_column_index, 0] +
                                                pr_others * (oldV[other_row_index_1, other_column_index_1, 0] +
                                                             oldV[other_row_index_2, other_column_index_2, 0] +
                                                             oldV[other_row_index_3, other_column_index_3, 0]))

                    # Update V(s) with the optimal value, given by the maximum value stored in Q where s is current state
                    # Update optimal_policy(s) with the optimal action, given by the index of the maximum value stored in Q
                    V[row_index,column_index,0] = max(Q.values())
                    optimal_policy[row_index,column_index,0] = max(Q, key=Q.get)
        number_iterations = number_iterations + 1
        if (abs(oldV[:,:,0] - V[:,:,0])<0.01).all():
            break
    return V, optimal_policy, number_iterations

In [10]:
# Obtain optimal value and policy
V, optimal_policy, number_iterations = value_iterations(V)

In [11]:
# Print optimal policy and number of iterations
optimal_policy = optimal_policy[:,:,0]
print('Optimal policy:')
print(optimal_policy)
print('Iterations:')
print(number_iterations)

Optimal policy:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 3. 3. 3. 3. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 3. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 0. 3. 3. 3.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 3. 3. 3. 3. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 3. 1. 1. 0. 3. 3. 3.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Iterations:
27
